In [25]:
import tensorflow as tf
from tensorflow import keras, sparse 
from keras import layers, applications
# from keras.preprocessing import image
from keras.applications.efficientnet import EfficientNetB0, EfficientNetB5, EfficientNetB7
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.densenet import DenseNet121
# from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator


import os 
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential

# Data preprocessing 
from keras.preprocessing.image import load_img, img_to_array
 

In [93]:

# subset IN (TRAINING, VALIDATION)
def getDataset(dataFolder, subset, imageSize = (224, 224), batchSize = 32):
    #      validation_split=0.05,
    # subset= subset,
    ds = keras.utils.image_dataset_from_directory(
      dataFolder,
      seed=123,
      image_size=imageSize,
      batch_size=batchSize)
    return train_ds


def configurePerformance(train_ds, val_ds): 
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    return train_ds, val_ds

def buildModel(dropoutRate, numClasses):
    inp = layers.Input(shape=inp_shape)
    baseModel = EfficientNetB0(weights="imagenet",
                       include_top = False) 
    baseModel.trainable = False 
    x = baseModel(inp, training=False)
    x =  layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.Dropout(dropoutRate, noise_shape=None, seed=None)(x)
    out = layers.Dense(numClasses,activation="softmax", name = "pred")(x)
    model = keras.Model(inp, out, name="B0")
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', 'precision', 'recall', 'f1'])
    
    return model 


In [94]:
EPOCHS = 10

trainingFolder = 'data/x5_RGB/train/RGB/'
testingFolder = 'data/x5_RGB/train/RGB/'

train_ds = getDataset(trainingFolder, "training")
val_ds =  getDataset(testingFolder, "validation")
model = buildModel(0.2, 7)


Found 6000 files belonging to 7 classes.
Found 6000 files belonging to 7 classes.


In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=EPOCHS
)


Epoch 1/10


ValueError: in user code:

    File "/Users/shreyesh/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/Users/shreyesh/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/shreyesh/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/Users/shreyesh/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 864, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/shreyesh/miniforge3/envs/env_tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 957, in compute_metrics
        self.compiled_metrics.update_state(y, 